# データサイエンス100本ノック（構造化データ加工編） - R

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとCSVファイルからのデータ読み込みを行います
- 利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（!マークに続けてOSコマンドを入力することで、任意のubuntu Linuxコマンドが入力可能）
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
# require('RPostgreSQL')
require('tidyr')
require('dplyr')
require('stringr')
require('caret')
require('lubridate')
require('rsample')
require('unbalanced')

# host <- 'db'
# port <- Sys.getenv()["PG_PORT"]
# dbname <- Sys.getenv()["PG_DATABASE"]
# user <- Sys.getenv()["PG_USER"]
# password <- Sys.getenv()["PG_PASSWORD"]

# con <- dbConnect(PostgreSQL(), host=host, port=port, dbname=dbname, user=user, password=password)
# df_customer <- dbGetQuery(con,"SELECT * FROM customer")
# df_category <- dbGetQuery(con,"SELECT * FROM category")
# df_product <- dbGetQuery(con,"SELECT * FROM product")
# df_receipt <- dbGetQuery(con,"SELECT * FROM receipt")
# df_store <- dbGetQuery(con,"SELECT * FROM store")
# df_geocode <- dbGetQuery(con,"SELECT * FROM geocode")

Loading required package: tidyr
Warning message:
"package 'tidyr' was built under R version 3.6.3"Loading required package: dplyr
Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: stringr
Warning message:
"package 'stringr' was built under R version 3.6.3"Loading required package: caret
Warning message:
"package 'caret' was built under R version 3.6.3"Loading required package: lattice
Loading required package: ggplot2
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date

Loading required package: rsample
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"

In [2]:
df_customer <- read.csv("./data/customer.csv", encoding="UTF-8")
head(df_customer, n = 5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS021313000114,"大野 あや子,1",女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS031415000172,"宇多田 貴美子,1",女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,
CS028811000001,"堀井 かおり,1",女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2


In [3]:
# df_category <- read.csv("./data/category.csv", encoding="UTF-8")
df_category <- read.csv("./data/category.csv", encoding="shift-jis") # エラーがでたので文字コードを変更
head(df_category, n = 5)

category_major_cd,category_major_name,category_medium_cd,category_medium_name,category_small_cd,category_small_name
4,惣菜,401,御飯類,40101,弁当類
4,惣菜,401,御飯類,40102,寿司類
4,惣菜,402,佃煮類,40201,魚介佃煮類
4,惣菜,402,佃煮類,40202,海草佃煮類
4,惣菜,402,佃煮類,40203,野菜佃煮類


In [4]:
df_product <- read.csv("./data/product.csv", encoding="UTF-8")
head(df_product, n = 5)

X.U.FEFF.product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
P040101001,4,401,40101,198,149
P040101002,4,401,40101,218,164
P040101003,4,401,40101,230,173
P040101004,4,401,40101,248,186
P040101005,4,401,40101,268,201


In [5]:
df_receipt <- read.csv("./data/receipt.csv", encoding="UTF-8")
head(df_receipt, n = 5)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [6]:
df_store <- read.csv("./data/store.csv", encoding="shift-jis") # 正常に読まなかったので文字コードを変換した
head(df_store, n = 5)

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679


In [7]:
df_geocode <- read.csv("./data/geocode.csv", encoding="UTF-8") # 時間がかかる, Python では一瞬で読めた
head(df_geocode, n = 5)

X.U.FEFF.postal_cd,prefecture,city,town,street,address,full_address,longitude,latitude
060-0000,"北海道,札幌市中央区",,,,北海道札幌市中央区,141.34103,43.05513,NA
064-0941,"北海道,札幌市中央区","旭ケ丘,",,"北海道札幌市中央区旭ケ丘,141.31972",43.04223,,NA,NA
060-0042,"北海道,札幌市中央区",大通西,,１丁目,北海道札幌市中央区大通西１丁目,141.35637,43.06102,NA
060-0042,"北海道,札幌市中央区",大通西,,２丁目,北海道札幌市中央区大通西２丁目,141.35445,43.06080,NA
060-0042,"北海道,札幌市中央区",大通西,,３丁目,北海道札幌市中央区大通西３丁目,141.35275,43.06086,NA


# 演習問題

---
> R-016: 店舗データフレーム（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [8]:
df_store %>% filter(grepl("[0-9]{3}-[0-9]{3}-[0-9]", tel_no))
# dplyr で部分一致する定番の方法ですね。
# 正規表現 で桁すうを表すときは {} の中に桁数を入れます。
# https://murashun.jp/article/programming/regular-expression.html

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.1180,35.63559,1698
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830
S13052,森野店,13,東京都,東京都町田市森野三丁目,トウキョウトマチダシモリノサンチョウメ,042-123-4030,139.4383,35.55293,1087
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341


---
> R-017: 顧客データフレーム（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭10件を全項目表示せよ。

In [9]:
df_tmp <- df_customer[order(df_customer$birth_day, decreasing = TRUE), ]
head(df_tmp, n=10)
# データフレームをある列でソートするときは[]の中にソート条件を order()で記入する。最後に「,」を忘れずに。
# http://taustation.com/r-dataframe-sort/

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
7512,CS001814000022,"鶴田 里穂,1",女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7,
18818,CS003813000014,"村山 菜々美,1",女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0,
36,CS003802000009,"大森 照生,0",男性,1930-01-23,89,214-0014,神奈川県川崎市多摩区登戸**********,S13003,20150216,0-00000000-0,
128,CS001812000030,"土屋 奈々,1",女性,1929-07-04,89,212-0024,神奈川県川崎市幸区塚越**********,S13001,20160821,0-00000000-0,
1122,CS008812000007,"高嶋 路子,1",女性,1930-01-09,89,213-0034,神奈川県川崎市高津区上作延**********,S13008,20150503,0-00000000-0,
1301,CS004814000023,"荒井 七世,1",女性,1930-02-21,89,176-0025,東京都練馬区中村南**********,S13004,20170315,7-20100922-5,
2964,CS005812000018,"砂川 瑠璃亜,1",女性,1929-08-22,89,167-0032,東京都杉並区天沼**********,S13005,20170916,0-00000000-0,
3447,CS038815000019,"田辺 怜奈,1",女性,1930-01-29,89,134-0084,東京都江戸川区東葛西**********,S13038,20150829,0-00000000-0,
3535,CS004813000015,"吉原 幸子,1",女性,1929-09-08,89,165-0035,東京都中野区白鷺**********,S13004,20150408,0-00000000-0,
3615,CS029803000002,"加藤 浩介,0",男性,1929-10-28,89,279-0041,千葉県浦安市堀江**********,S12029,20150701,2-20100329-5,


In [10]:
# # sort()関数を使うと
# # https://note.com/marginal_ard30/n/nd2e1632b5874
# df_tmp2 <- df_customer[sort(df_customer$birth_day, decreasing = FALSE,index=T)$ix,])
# head(df_tmp2, n=10)

---
> R-018: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭10件を全項目表示せよ。

In [11]:
df_tmp <- df_customer[order(df_customer$birth_day, decreasing = FALSE), ]
head(df_tmp, n=10)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
3671,CS025115000002,"小柳 夏希,1",女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D,
10746,CS002113000009,"柴田 真悠子,1",女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0,
19812,CS004115000014,"松井 京子,1",女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1,
1399,CS040113000005,"尾崎 幸子,1",女性,2006-07-07,12,194-0001,東京都町田市つくし野**********,S14040,20151130,0-00000000-0,
2243,CS040112000011,"柳川 陽子,1",女性,2006-05-21,12,194-0014,東京都町田市高ヶ坂**********,S14040,20180927,0-00000000-0,
10951,CS016114000006,"高山 涼子,1",女性,2006-08-12,12,184-0011,東京都小金井市東町**********,S13016,20150514,9-20090628-7,
11606,CS004115000022,"竹村 美帆,1",女性,2006-08-27,12,165-0031,東京都中野区上鷺宮**********,S13004,20161113,0-00000000-0,
13690,CS023114000002,"葛西 茜,1",女性,2006-10-10,12,144-0046,東京都大田区東六郷**********,S14023,20150117,A-20100228-D,
14918,CS002112000013,"森下 さやか,1",女性,2006-11-10,12,190-0001,東京都立川市若葉町**********,S13002,20171106,0-00000000-0,
15098,CS014113000008,"矢口 莉緒,1",女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6,


---
> R-019: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [12]:
head(df_receipt)

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138


In [13]:
df_receipt %>%
    mutate(ranking = min_rank(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 
# こんな処理が実質１行で書けるのは dplyr の威力ですね。
# mutate() はデータセットに新たに変数を追加する関数
# https://kazutan.github.io/kazutanR/hands_on_170730/mutate.html
# arrange() は行をなら得変える
# https://qiita.com/matsuou1/items/e995da273e3108e2338e
# slice()　指定した行を切り出す。
# https://hikaru1122.hatenadiary.jp/entry/2015/05/05/165831

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking
20190903,1283472000,S14011,1192,2,CS011415000006,P080104055,1,10925,1
20190304,1267660800,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2
20170921,1221955200,S14028,1192,2,CS028605000002,P090907210,1,5780,3
20170721,1216598400,S13015,1112,1,CS015515000034,P090907208,1,5480,4
20190103,1262476800,S14021,1162,2,ZZ000000000000,P090907208,1,5480,4
20190724,1279929600,S13009,1192,2,ZZ000000000000,P090907209,1,5480,4
20180621,1245542400,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7
20190209,1265673600,S14021,1142,2,CS021515000089,P090907206,1,5440,7
20181219,1261180800,S13015,1132,1,CS015515000083,P071301009,1,5280,9
20180603,1243987200,S13017,1192,1,CS017414000114,P071301009,1,5280,9


---
> R-020: レシート明細データフレーム（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭10件を抽出せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [14]:
df_receipt %>%
    mutate(ranking = row_number(desc(amount))) %>%
    arrange(ranking) %>%
    slice(1:10) 
# mutate()関数の引数で使うランキング関数
# https://qiita.com/matsuou1/items/db6e8c48fcfd791dd876

sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,ranking
20190903,1283472000,S14011,1192,2,CS011415000006,P080104055,1,10925,1
20190304,1267660800,S14046,1152,2,ZZ000000000000,P090302118,1,6800,2
20170921,1221955200,S14028,1192,2,CS028605000002,P090907210,1,5780,3
20170721,1216598400,S13015,1112,1,CS015515000034,P090907208,1,5480,4
20190103,1262476800,S14021,1162,2,ZZ000000000000,P090907208,1,5480,5
20190724,1279929600,S13009,1192,2,ZZ000000000000,P090907209,1,5480,6
20180621,1245542400,S13004,1172,2,ZZ000000000000,P090907206,1,5440,7
20190209,1265673600,S14021,1142,2,CS021515000089,P090907206,1,5440,8
20181219,1261180800,S13015,1132,1,CS015515000083,P071301009,1,5280,9
20180603,1243987200,S13017,1192,1,CS017414000114,P071301009,1,5280,10


### 主なランキング関数  
<table border="1" align="left" widtd=600>
<tr>
    <th widtd=100>関数</th>
    <th>説明</th>
<tr>
    <td>row_number</td>
    <td>昇順にランキングを付ける。同じ値がある場合は、最初に来た方を優先</td>
<tr>
    <td>min_rank</td>
    <td>昇順にランキングを付ける。同じ値がある場合は、同じ順位を付ける。gapあり</td>
<tr>
    <td>dense_rank</td>
    <td>昇順にランキングを付ける。同じ値がある場合は、同じ順位を付ける。gapなし</td>
<tr>
    <td>percent_rank</td>
    <td>min_rankを0～1にリスケールしたもの</td>
<tr>
    <td>cume_dist</td>
    <td>累積割合。percent_rankの累積和ではない。</td>
<tr>
    <td>ntile</td>
    <td>n個の群に分割する</td>
</table>